In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from doc_scraper import fetch_doc_links, fetch_doc_contents, is_same_domain
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')
base_url = os.getenv('OR_BASE_URL')
MODEL = os.getenv('MODEL')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI(base_url=base_url, api_key=api_key)

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
# Test fetching links
links = fetch_doc_links("https://react.dev/reference/react")
print(f"Found {len(links)} links")
links[:5]  # Show first 5

Found 181 links


[{'text': 'React', 'url': 'https://react.dev/'},
 {'text': 'v19.2', 'url': 'https://react.dev/versions'},
 {'text': 'Learn', 'url': 'https://react.dev/learn'},
 {'text': 'Reference', 'url': 'https://react.dev/reference/react'},
 {'text': 'Community', 'url': 'https://react.dev/community'}]

## First step: Have GPT analyze the documentation structure

### Use a call to GPT to read the links on a documentation page, and respond in structured JSON.
It should decide which links are most relevant for learning, and identify their priority level.
This ensures we fetch the most important educational content.

In [4]:
doc_link_system_prompt = """
You are an expert at analyzing technical documentation websites.
You are provided with a list of links found on a documentation page.
You should identify links that are most relevant for learning the technology, such as:
- Core concepts and fundamentals
- API reference pages
- Tutorial or guide pages
- Getting started pages
- Important feature documentation

Exclude: Navigation links, external links, changelog, blog posts, community forums.

Respond in JSON format:
{
    "links": [
        {"type": "getting started", "url": "https://full.url/here", "priority": "high"},
        {"type": "core concept", "url": "https://another.url/here", "priority": "medium"}
    ]
}

Priority should be: "high" for essential concepts, "medium" for important features, "low" for advanced topics.
"""

In [5]:
def get_doc_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the documentation page {url}.
Please identify which links are most relevant for learning this technology.
Focus on documentation that teaches concepts, APIs, and practical usage.
Respond with the full https URL in JSON format.

Links:

"""
    links = fetch_doc_links(url)
    # Filter to same domain and format nicely
    base_domain = url
    for link in links[:50]:  # Limit to first 50 to avoid token limits
        if is_same_domain(link["url"], url):
            user_prompt += f"{link['text']}: {link['url']}\n"
    
    return user_prompt

In [6]:
print(get_doc_links_user_prompt("https://react.dev/reference/react"))


Here is the list of links on the documentation page https://react.dev/reference/react.
Please identify which links are most relevant for learning this technology.
Focus on documentation that teaches concepts, APIs, and practical usage.
Respond with the full https URL in JSON format.

Links:

React: https://react.dev/
v19.2: https://react.dev/versions
Learn: https://react.dev/learn
Reference: https://react.dev/reference/react
Community: https://react.dev/community
Blog: https://react.dev/blog
Overview: https://react.dev/reference/react
Hooks: https://react.dev/reference/react/hooks
useActionState: https://react.dev/reference/react/useActionState
useCallback: https://react.dev/reference/react/useCallback
useContext: https://react.dev/reference/react/useContext
useDebugValue: https://react.dev/reference/react/useDebugValue
useDeferredValue: https://react.dev/reference/react/useDeferredValue
useEffect: https://react.dev/reference/react/useEffect
useEffectEvent: https://react.dev/reference

In [7]:
def select_relevant_doc_links(url, max_links=5):
    print(f"Analyzing documentation structure at {url}...")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": doc_link_system_prompt},
            {"role": "user", "content": get_doc_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    
    # Sort by priority and limit
    priority_order = {"high": 0, "medium": 1, "low": 2}
    sorted_links = sorted(
        links['links'], 
        key=lambda x: priority_order.get(x.get('priority', 'low'), 3)
    )[:max_links]
    
    print(f"Selected {len(sorted_links)} key documentation pages")
    return {"links": sorted_links}

In [8]:
select_relevant_doc_links("https://react.dev/reference/react", max_links=3)

Analyzing documentation structure at https://react.dev/reference/react...
Selected 3 key documentation pages


{'links': [{'type': 'getting started',
   'url': 'https://react.dev/learn',
   'priority': 'high'},
  {'type': 'core concept',
   'url': 'https://react.dev/reference/react',
   'priority': 'high'},
  {'type': 'api reference',
   'url': 'https://react.dev/reference/react/hooks',
   'priority': 'high'}]}

In [9]:
def fetch_doc_page_and_links(url, max_links=5):
    print(f"Fetching documentation content from {url}...")
    contents = fetch_doc_contents(url)
    relevant_links = select_relevant_doc_links(url, max_links)
    
    result = f"## Main Documentation Page:\n\n{contents}\n\n## Related Documentation Pages:\n"
    
    for i, link in enumerate(relevant_links['links'], 1):
        print(f"Fetching page {i}/{len(relevant_links['links'])}: {link['type']}")
        result += f"\n\n### {link['type'].title()} (Priority: {link.get('priority', 'N/A')})\n"
        result += f"URL: {link['url']}\n\n"
        result += fetch_doc_contents(link["url"], max_chars=2500)
    
    return result

In [10]:
content = fetch_doc_page_and_links("https://react.dev/reference/react", max_links=2)
print(f"Total content length: {len(content)} characters")

Fetching documentation content from https://react.dev/reference/react...
Analyzing documentation structure at https://react.dev/reference/react...
Selected 2 key documentation pages
Fetching page 1/2: getting started
Fetching page 2/2: core concept
Total content length: 7709 characters


## Second step: Generate beginner-friendly teaching materials!

Assemble all the documentation details into a comprehensive teaching guide with GPT.

In [11]:
teaching_guide_system_prompt = """
You are an expert technical educator who excels at breaking down complex documentation 
into clear, beginner-friendly teaching materials.

Your task is to analyze technical documentation and create a comprehensive teaching guide that includes:

1. **Overview**: A clear, simple explanation of what this technology is and why it matters
2. **Key Concepts**: The most important concepts explained in plain language with analogies
3. **Core Features**: Main features and their practical applications
4. **Code Examples**: Simple, commented code examples that demonstrate key concepts
5. **Learning Path**: Suggested order for learning topics (beginner → intermediate → advanced)
6. **Common Pitfalls**: Things beginners often struggle with and how to avoid them
7. **Practice Projects**: 2-3 small project ideas to practice the concepts

Format your response in clear markdown with headers, bullet points, and code blocks.
Make it engaging and easy to follow for someone new to this technology.
Use emojis sparingly but effectively to make it more engaging.
"""

In [12]:
def get_teaching_guide_user_prompt(tech_name, url, max_links=5):
    user_prompt = f"""
You are analyzing documentation for: {tech_name}
Main documentation URL: {url}

Below is the content from the main documentation page and several related important pages.
Use this information to create a comprehensive, beginner-friendly teaching guide.

"""
    user_prompt += fetch_doc_page_and_links(url, max_links)
    
    # Truncate if too long (keep it under 12k chars to leave room for response)
    if len(user_prompt) > 12_000:
        user_prompt = user_prompt[:12_000] + "\n\n[Content truncated for length]"
    
    return user_prompt

In [13]:
def create_teaching_guide(tech_name, url, max_links=3):
    print(f"Creating teaching guide for {tech_name}...")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": teaching_guide_system_prompt},
            {"role": "user", "content": get_teaching_guide_user_prompt(tech_name, url, max_links)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [14]:
create_teaching_guide("React Hooks", "https://react.dev/reference/react", max_links=3)

Creating teaching guide for React Hooks...
Fetching documentation content from https://react.dev/reference/react...
Analyzing documentation structure at https://react.dev/reference/react...
Selected 3 key documentation pages
Fetching page 1/3: getting started
Fetching page 2/3: core concept
Fetching page 3/3: core concept


## React Hooks: Beginner-Friendly Teaching Guide

### Overview

React Hooks are a powerful feature in React that allow you to use state and other React capabilities in functional components. Before hooks, these features were only available in class components. Hooks make it easier to reuse code, manage state, and perform side effects, all while keeping your components clean and concise. They are a game-changer for anyone learning React, as they simplify the process of building interactive and dynamic user interfaces.

### Key Concepts

- **Hooks**: Think of hooks like special functions that let you "hook into" React features. They start with the word `use` (e.g., `useState`, `useEffect`).

- **State**: Imagine state as a memory for your component. It lets your component remember things, like user input or a counter value, and change them over time.

- **Effects**: Effects are like side tasks your component can do, such as fetching data from a server or updating the document title. They run after your component has rendered.

- **Context**: Context is like a secret channel your component can listen to, allowing it to receive information from distant "ancestors" without having to pass props down through every level.

### Core Features

- **`useState`**: Allows you to add state to a functional component.
  ```javascript
  function Counter() {
    const [count, setCount] = useState(0);
    return (
      <div>
        <p>You clicked {count} times</p>
        <button onClick={() => setCount(count + 1)}>Click me</button>
      </div>
    );
  }
  ```

- **`useEffect`**: Lets you perform side effects in a functional component.
  ```javascript
  function DataFetcher({ url }) {
    useEffect(() => {
      let isMounted = true;
      fetch(url)
        .then(response => response.json())
        .then(data => {
          if (isMounted) {
            // Update state with fetched data
          }
        });
      return () => { isMounted = false; };
    }, [url]);
    return <div>Loading...</div>;
  }
  ```

- **`useContext`**: Enables a component to use the nearest context value.
  ```javascript
  // Create a context
  const ThemeContext = React.createContext('light');

  function ThemedButton() {
    const theme = useContext(ThemeContext);
    return <button className={theme}>I am styled by theme context!</button>;
  }
  ```

- **`useRef`**: Allows you to reference a value that persists across renders.
  ```javascript
  function TextInputWithFocusButton() {
    const inputEl = useRef(null);
    const onButtonClick = () => {
      // `current` points to the mounted text input element
      inputEl.current.focus();
    };
    return (
      <div>
        <input ref={inputEl} type="text" />
        <button onClick={onButtonClick}>Focus the input</button>
      </div>
    );
  }
  ```

### Learning Path

1. **Beginner**
   - Understand what functional components are.
   - Learn how to use `useState` for basic state management.
   - Explore how to use `useEffect` for simple side effects.

2. **Intermediate**
   - Master `useContext` for global state management.
   - Learn about custom hooks and how to create your own.
   - Explore `useReducer` for more complex state logic.

3. **Advanced**
   - Understand performance optimizations with `useMemo` and `useCallback`.
   - Learn about `useLayoutEffect` and when to use it.
   - Explore advanced patterns with hooks, such as `useReducer` with context.

### Common Pitfalls

- **Dependency Array in `useEffect`**: Forgetting to include all necessary variables in the dependency array can lead to stale closures. Always ensure your dependencies are correct.
  ```javascript
  useEffect(() => {
    document.title = `You clicked ${count} times`;
  }, [count]); // ← Include 'count' in the dependency array
  ```

- **Calling Hooks Conditionally**: Hooks should always be called unconditionally at the top level of a component. This means no conditionals, loops, or nested functions.
  ```javascript
  if (isLoggedIn) {
    const user = useUser(); // 🚨 Hook call inside a conditional
  }
  ```

- **State Mutations**: Directly mutating state (e.g., `state.push(newItem)`) can lead to unexpected behaviors. Always update state immutably using functions like `setState`.

### Practice Projects

1. **To-Do List App**: Create a simple to-do list where you can add, remove, and mark items as complete. Use `useState` for managing the list of todos and their statuses.
   ```javascript
   function TodoApp() {
     const [todos, setTodos] = useState([]);
     const [text, setText] = useState('');

     const handleAddTodo = () => {
       setTodos([...todos, { text, completed: false }]);
       setText('');
     };

     return (
       <div>
         <input
           value={text}
           onChange={(e) => setText(e.target.value)}
         />
         <button onClick={handleAddTodo}>Add Todo</button>
         <ul>
           {todos.map((todo, index) => (
             <li key={index}>{todo.text}</li>
           ))}
         </ul>
       </div>
     );
   }
   ```

2. **Weather App**: Build a weather app that fetches and displays weather data from an API. Use `useEffect` to fetch data when the component mounts and `useState` to store the fetched data.
   ```javascript
   function WeatherApp({ city }) {
     const [weather, setWeather] = useState(null);

     useEffect(() => {
       fetch(`https://api.weatherapi.com/v1/current.json?key=YOUR_API_KEY&q=${city}`)
         .then(response => response.json())
         .then(data => setWeather(data));
     }, [city]);

     if (!weather) return <div>Loading...</div>;

     return (
       <div>
         <h1>{weather.location.name}</h1>
         <p>{weather.current.temp_c}°C</p>
       </div>
     );
   }
   ```

3. **Theme Toggler**: Create a theme toggler component that switches between light and dark themes. Use `useContext` to provide the theme context to the entire application.
   ```javascript
   const ThemeContext = React.createContext('light');

   function ThemeToggler() {
     const [theme, setTheme] = useState('light');
     return (
       <ThemeContext.Provider value={theme}>
         <button onClick={() => setTheme(theme === 'light' ? 'dark' : 'light')}>
           Toggle Theme
         </button>
         <ThemedComponent />
       </ThemeContext.Provider>
     );
   }

   function ThemedComponent() {
     const theme = useContext(ThemeContext);
     return <div className={theme}>I am styled by theme context!</div>;
   }
   ```

By following this guide, you'll be well on your way to mastering React Hooks and building dynamic, interactive web applications with ease!

'## React Hooks: Beginner-Friendly Teaching Guide\n\n### Overview\n\nReact Hooks are a powerful feature in React that allow you to use state and other React capabilities in functional components. Before hooks, these features were only available in class components. Hooks make it easier to reuse code, manage state, and perform side effects, all while keeping your components clean and concise. They are a game-changer for anyone learning React, as they simplify the process of building interactive and dynamic user interfaces.\n\n### Key Concepts\n\n- **Hooks**: Think of hooks like special functions that let you "hook into" React features. They start with the word `use` (e.g., `useState`, `useEffect`).\n\n- **State**: Imagine state as a memory for your component. It lets your component remember things, like user input or a counter value, and change them over time.\n\n- **Effects**: Effects are like side tasks your component can do, such as fetching data from a server or updating the documen

## Streaming version for better user experience

With a small adjustment, we can stream back the results from OpenAI,
with the familiar typewriter animation

In [15]:
def stream_teaching_guide(tech_name, url, max_links=3):
    print(f"Creating teaching guide for {tech_name}...")
    print("=" * 60)
    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": teaching_guide_system_prompt},
            {"role": "user", "content": get_teaching_guide_user_prompt(tech_name, url, max_links)}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
    
    return response

In [16]:
result = stream_teaching_guide("React", "https://react.dev/reference/react", max_links=3)

Creating teaching guide for React...
Fetching documentation content from https://react.dev/reference/react...
Analyzing documentation structure at https://react.dev/reference/react...
Selected 3 key documentation pages
Fetching page 1/3: getting started
Fetching page 2/3: core concept
Fetching page 3/3: core concept


# React Teaching Guide

Welcome to the React Teaching Guide! React is a popular JavaScript library for building user interfaces, particularly single-page applications. It allows developers to create large web applications that can update and render efficiently in response to data changes. React is maintained by Facebook and a community of individual developers and companies. Let's dive into the key concepts and features that make React powerful and flexible.

## 1. Overview

React is a library for building user interfaces. It is designed to make the development of complex UIs more manageable by breaking them down into smaller, reusable components. React components are JavaScript functions or classes that return a piece of the UI. This modular approach makes it easier to develop, understand, and maintain your application.

### Why React Matters
- **Component-Based Architecture**: React encourages the creation of reusable components, making it easier to manage and update parts of your application.
- **Efficient Updates**: React uses a virtual DOM to minimize direct manipulations of the browser’s DOM, resulting in faster and more efficient updates.
- **Large Community and Ecosystem**: React has a vast ecosystem of libraries and tools, making it easier to find solutions and get help.

## 2. Key Concepts

### Components
Components are the building blocks of a React application. They are JavaScript functions or classes that return a piece of the UI.

- **Function Components**: Simple functions that return JSX.
  ```javascript
  function MyButton() {
    return <button>I'm a button</button>;
  }
  ```
- **Class Components**: ES6 classes that extend `React.Component`.
  ```javascript
  class MyButton extends React.Component {
    render() {
      return <button>I'm a button</button>;
    }
  }
  ```

### JSX (JavaScript XML)
JSX is a syntax extension that allows you to write HTML-like code within JavaScript. It is optional but widely used for its convenience.

- **Example**: Using JSX to define a component.
  ```javascript
  function App() {
    return (
      <div>
        <h1>Welcome to my app</h1>
        <MyButton />
      </div>
    );
  }
  ```

### State and Props
- **State**: Internal data that a component manages. It can change over time and triggers a re-render when updated.
  ```javascript
  function Counter() {
    const [count, setCount] = React.useState(0);
    return (
      <div>
        <p>You clicked {count} times</p>
        <button onClick={() => setCount(count + 1)}>
          Click me
        </button>
      </div>
    );
  }
  ```
- **Props**: Data passed from parent to child components. They are read-only and should not be modified by the child component.

### Hooks
Hooks are functions that let you use state and other React features without writing a class. They are a way to "hook into" React's features from functional components.

- **useState**: Allows you to add state to a functional component.
  ```javascript
  const [count, setCount] = React.useState(0);
  ```
- **useEffect**: Lets you perform side effects in function components.
  ```javascript
  React.useEffect(() => {
    document.title = `You clicked ${count} times`;
  }, [count]);
  ```

## 3. Core Features

### Declarative
React uses a declarative approach to building UIs. You describe what the UI should look like for a given state, and React efficiently updates and renders just the right components when your data changes.

### Component-Based
Everything in React is a component. Components are self-contained and can manage their own state, making them easy to test and reuse.

### Learn Once, Write Anywhere
React can be used to build applications for the web, mobile, and even desktop. It provides a consistent development experience across different platforms.

## 4. Code Examples

### Creating a Simple Component
```javascript
// MyButton.js
function MyButton() {
  return <button>I'm a button</button>;
}

export default MyButton;
```

### Using State in a Component
```javascript
// Counter.js
import React, { useState } from 'react';

function Counter() {
  const [count, setCount] = useState(0);

  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={() => setCount(count + 1)}>
        Click me
      </button>
    </div>
  );
}

export default Counter;
```

### Conditional Rendering
```javascript
function Greeting({ isLoggedIn }) {
  if (isLoggedIn) {
    return <h1>Welcome back!</h1>;
  }
  return <h1>Please sign up.</h1>;
}
```

### List Rendering
```javascript
function NumberList({ numbers }) {
  return (
    <ul>
      {numbers.map((number) => (
        <li key={number.toString()}>
          {number}
        </li>
      ))}
    </ul>
  );
}
```

## 5. Learning Path

### Beginner
1. **Introduction to React**: Understand what React is and why it's used.
2. **JSX Basics**: Learn how to write JSX and its benefits.
3. **Components**: Create simple function and class components.
4. **Props**: Pass data from parent to child components.

### Intermediate
1. **State Management**: Use `useState` to manage component state.
2. **Hooks**: Explore other hooks like `useEffect`, `useContext`, and `useReducer`.
3. **Conditional Rendering**: Render components conditionally based on state or props.
4. **List and Key**: Render lists and understand the importance of keys.

### Advanced
1. **Context API**: Manage global state without prop drilling.
2. **Custom Hooks**: Create your own hooks to reuse logic across components.
3. **Performance Optimization**: Understand memoization and other performance techniques.
4. **React Router**: Implement routing in your React applications.

## 6. Common Pitfalls

### Pitfall 1: Incorrect Component Naming
- **Issue**: React components must start with a capital letter, while HTML tags must be lowercase.
- **Solution**: Always start component names with a capital letter.
  ```javascript
  function MyComponent() { /* ... */ }
  ```

### Pitfall 2: Improper State Update
- **Issue**: Updating state directly can lead to unexpected behavior.
- **Solution**: Use the state setter function to update state.
  ```javascript
  setCount(count + 1); // Correct
  count = count + 1; // Incorrect
  ```

### Pitfall 3: Missing Keys in Lists
- **Issue**: Not providing a unique `key` prop to list items can cause performance issues and bugs.
- **Solution**: Always provide a unique `key` prop to list items.
  ```javascript
  {numbers.map(number => (
    <li key={number.toString()}>{number}</li>
  ))}
  ```

## 7. Practice Projects

### Project 1: Todo List Application
- **Description**: Build a simple todo list where users can add, remove, and mark todos as complete.
- **Features to Implement**:
  - Add a new todo item.
  - Delete a todo item.
  - Mark a todo as complete.
  - Persist data using local storage.

### Project 2: Weather Application
- **Description**: Create a weather application that fetches and displays current weather data for a given location.
- **Features to Implement**:
  - Fetch weather data from an API.
  - Display the current temperature, weather condition, and location.
  - Handle API errors and loading states.

### Project 3: Blog Platform
- **Description**: Develop a basic blog platform where users can create, read, update, and delete blog posts.
- **Features to Implement**:
  - Create a new blog post with a title and content.
  - Display a list of blog posts.
  - Allow users to edit and delete their posts.
  - Implement basic authentication for user accounts.

---

## Try with different documentation sites

Uncomment to try different docs:

In [17]:
# Python asyncio
# stream_teaching_guide("Python Asyncio", "https://docs.python.org/3/library/asyncio.html", max_links=2)

# FastAPI
# stream_teaching_guide("FastAPI", "https://fastapi.tiangolo.com/", max_links=3)

# Vue.js
# stream_teaching_guide("Vue.js", "https://vuejs.org/guide/introduction.html", max_links=3)

## Advanced - Generate Practice Quiz Questions

Test understanding with AI-generated quiz questions based on the documentation

In [18]:
quiz_system_prompt = """
You are an expert at creating educational assessment questions.
Based on the technical documentation provided, create 5 multiple-choice quiz questions 
that test understanding of key concepts.

Format each question as:
**Question X:** [Question text]
- A) [Option]
- B) [Option]
- C) [Option]
- D) [Option]

**Answer:** [Correct letter] - [Brief explanation]

Make questions practical and test real understanding, not just memorization.
"""

def create_quiz(tech_name, url):
    print(f"Creating quiz for {tech_name}...")
    user_prompt = f"Create quiz questions based on this documentation:\n\n"
    user_prompt += fetch_doc_page_and_links(url, max_links=2)
    user_prompt = user_prompt[:8000]  # Limit length
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": quiz_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [19]:
create_quiz("React Hooks", "https://react.dev/reference/react")

Creating quiz for React Hooks...
Fetching documentation content from https://react.dev/reference/react...
Analyzing documentation structure at https://react.dev/reference/react...
Selected 2 key documentation pages
Fetching page 1/2: getting started
Fetching page 2/2: core concept


**Question 1:** What is the primary purpose of the React Reference Overview documentation?
- A) To provide a beginner's guide to learning React
- B) To offer detailed reference documentation for working with React
- C) To explain the history and evolution of React
- D) To list all available third-party libraries for React

**Answer:** B) To offer detailed reference documentation for working with React - This section is designed to provide in-depth information for developers already familiar with React, helping them understand and utilize its various features and components effectively.

---

**Question 2:** Which of the following is NOT a subcategory under the "React" section of the documentation?
- A) Hooks
- B) Components
- C) Directives
- D) Styles

**Answer:** D) Styles - The "React" section is broken down into Hooks, Components, APIs, and Directives, but does not have a subcategory specifically for "Styles."

---

**Question 3:** What is the purpose of the "React Compiler" in the context of the provided documentation?
- A) To automatically generate boilerplate code for new projects
- B) To automatically memoize React components and values
- C) To translate React code into different programming languages
- D) To optimize the performance of third-party libraries

**Answer:** B) To automatically memoize React components and values - The React Compiler is described as a build-time optimization tool that automatically memoizes components and values, improving performance.

---

**Question 4:** According to the "Rules of React," which of the following statements about components and hooks is correct?
- A) Components and hooks can be impure functions
- B) Components and hooks must be pure to ensure high-quality applications
- C) React automatically optimizes only impure components and hooks
- D) Purity in components and hooks is not necessary for optimization

**Answer:** B) Components and hooks must be pure to ensure high-quality applications - Purity in components and hooks makes the code easier to understand, debug, and allows React to optimize them correctly.

---

**Question 5:** Which section of the documentation would you refer to if you wanted to learn about rendering React components on the server?
- A) React DOM Client APIs
- B) React DOM Server APIs
- C) React DOM Static APIs
- D) React Compiler Configuration

**Answer:** B) React DOM Server APIs - The "react-dom/server" APIs are specifically designed to render React components to HTML on the server, which corresponds to server-side rendering.

'**Question 1:** What is the primary purpose of the React Reference Overview documentation?\n- A) To provide a beginner\'s guide to learning React\n- B) To offer detailed reference documentation for working with React\n- C) To explain the history and evolution of React\n- D) To list all available third-party libraries for React\n\n**Answer:** B) To offer detailed reference documentation for working with React - This section is designed to provide in-depth information for developers already familiar with React, helping them understand and utilize its various features and components effectively.\n\n---\n\n**Question 2:** Which of the following is NOT a subcategory under the "React" section of the documentation?\n- A) Hooks\n- B) Components\n- C) Directives\n- D) Styles\n\n**Answer:** D) Styles - The "React" section is broken down into Hooks, Components, APIs, and Directives, but does not have a subcategory specifically for "Styles."\n\n---\n\n**Question 3:** What is the purpose of the "R

## Complete pipeline function

Generate a complete learning package with guide + quiz in one call

In [20]:
def generate_complete_learning_package(tech_name, url, max_links=3):
    """Generate a complete learning package: guide + quiz"""
    print(f"\n{'='*60}")
    print(f"GENERATING COMPLETE LEARNING PACKAGE FOR: {tech_name}")
    print(f"{'='*60}\n")
    
    # Generate teaching guide
    print("\n📚 Creating Teaching Guide...\n")
    guide = stream_teaching_guide(tech_name, url, max_links)
    
    print("\n\n" + "="*60)
    print("\n❓ Creating Practice Quiz...\n")
    quiz = create_quiz(tech_name, url)
    
    print("\n" + "="*60)
    print("✅ Complete learning package generated!")
    print("="*60)
    
    return {"guide": guide, "quiz": quiz}

In [21]:
package = generate_complete_learning_package(
    "React useState Hook", 
    "https://react.dev/reference/react/useState",
    max_links=2
)


GENERATING COMPLETE LEARNING PACKAGE FOR: React useState Hook


📚 Creating Teaching Guide...

Creating teaching guide for React useState Hook...
Fetching documentation content from https://react.dev/reference/react/useState...
Analyzing documentation structure at https://react.dev/reference/react/useState...
Selected 2 key documentation pages
Fetching page 1/2: core concept
Fetching page 2/2: getting started


# Comprehensive Guide to React's useState Hook

Welcome to your beginner-friendly guide to the `useState` Hook in React! 🌟

## Overview

React's `useState` is a super handy tool that lets you add **state** to your components. Think of state as a magical box where you store important information that can change over time. When the data in this box changes, your component automatically updates to reflect those changes. This is like having a dynamic and responsive UI that adjusts to user interactions!

## Key Concepts

### 1. **State**

**Analogy:** Imagine state as a notebook where you keep tracked notes. Whenever you update your notes, the notebook changes. In React, state is like this notebook for your component.

### 2. **Hooks**

**Analogy:** Hooks are like special superpowers that let you use state and other react features in functional components. `useState` is one of these superpowers.

### 3. **Component**

**Analogy:** Think of a component as a Lego block. It's a reusable piece of your UI that has its own logic and appearance.

## Core Features

### 1. **Adding State**

You add state to a component using the `useState` function.

```jsx
import { useState } from 'react';

function MyComponent() {
  const [age, setAge] = useState(28);
  return <div>Your age is {age}</div>;
}
```

### 2. **Updating State**

You can update state using the `set` function returned by `useState`.

```jsx
function handleClick() {
  setAge(29); // Direct update
  setAge(prevAge => prevAge + 1); // Update based on previous state
}
```

### 3. **Initializer Function**

You can pass a function as the initial state to create complex initial values.

```jsx
const [todos, setTodos] = useState(() => createTodos());
```

## Code Examples

### Basic Usage

```jsx
import { useState } from 'react';

function Counter() {
  const [count, setCount] = useState(0);

  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={() => setCount(count + 1)}>
        Click me
      </button>
    </div>
  );
}
```

### Complex Initial State

```jsx
import { useState } from 'react';

function TodoApp() {
  const [todos, setTodos] = useState(() => createTodos());

  // Function to create initial todos
  function createTodos() {
    return [{ text: 'Learn React', completed: false }];
  }

  return (
    <div>
      <h1>Todo List</h1>
      <ul>
        {todos.map((todo, index) => (
          <li key={index}>{todo.text}</li>
        ))}
      </ul>
    </div>
  );
}
```

## Learning Path

1. **Beginner:**
   - Understand what state is and why it's important.
   - Learn how to declare state using `useState`.
   - Practice updating state directly.

2. **Intermediate:**
   - Learn to update state based on the previous state.
   - Understand the initializer function and when to use it.
   - Explore state in nested components.

3. **Advanced:**
   - Manage complex state structures.
   - Optimize state updates for performance.
   - Integrate state management with other hooks like `useEffect`.

## Common Pitfalls

### 1. **State Not Updating**

**Issue:** You update the state, but the component doesn't re-render.

**Solution:** Ensure you are using the `set` function correctly and that the state update is not being ignored.

### 2. **Too Many Re-renders**

**Issue:** Your component re-renders infinitely.

**Solution:** Check for any state updates inside render that might be causing this loop. Move state updates to event handlers.

### 3. **Initializer Function Runs Twice**

**Issue:** In development mode, your initializer function runs twice.

**Solution:** This is normal in Strict Mode. Ensure your function is pure and does not have side effects.

## Practice Projects

### 1. **Simple Counter**

Create a counter component that increments a number each time a button is clicked.

### 2. **Todo List Application**

Build a todo list where users can add, remove, and mark todos as completed.

### 3. **Form Handling**

Create a form with inputs for name and age. Use state to manage form data and display it below the form.

Happy coding! 🚀




❓ Creating Practice Quiz...

Creating quiz for React useState Hook...
Fetching documentation content from https://react.dev/reference/react/useState...
Analyzing documentation structure at https://react.dev/reference/react/useState...
Selected 2 key documentation pages
Fetching page 1/2: getting started
Fetching page 2/2: api reference


RateLimitError: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'cognitivecomputations/dolphin-mistral-24b-venice-edition:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'Venice'}}, 'user_id': 'user_2xJGnUD1uqIdThXpcPHNwMXzMpq'}

## Real-World Business Applications

This documentation teacher system can be applied to:

1. **Corporate Training**: Automatically generate training materials for internal tools and frameworks
2. **Developer Onboarding**: Create personalized learning paths for new team members
3. **Educational Platforms**: Build adaptive learning content from any technical documentation
4. **Technical Writing**: Generate first drafts of tutorials and guides from API docs
5. **Documentation Enhancement**: Create supplementary learning materials for existing docs

## Experiment With Different Docs

Try it with:
- Framework docs (React, Vue, Angular, Svelte)
- Language docs (Python, JavaScript, Rust)
- Library docs (pandas, NumPy, TensorFlow)
- API documentation (Stripe, Twilio, AWS)

## Next Steps

Consider enhancing this with:
- Code example validation
- Interactive code playgrounds
- Video script generation
- Difficulty level customization
- Multi-language support